In [1]:
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
from nptyping import Array
import matplotlib.pyplot as plt

from typing import List, Dict
import os
import glob

## GPU Setting

In [2]:
tf.debugging.set_log_device_placement(True)
gpus = tf.config.experimental.list_physical_devices('GPU')

## Hyperparameter

In [3]:
LATENT_DIM = 2048
INPUT_SHAPE = (16, 512, 512, 3) # frames, width, height, channels

In [4]:
class ResBLockConv3D(keras.models.Model):
    def __init__(self, filters, conv_num, kernel_size=(3, 3, 3)):
        super(ResBLockConv3D, self).__init__()
        self.convs: List[layers.Layer] = []
        for i in range(conv_num):
            conv = layers.Conv3D(filters=filters,
                                 kernel_size=kernel_size,
                                 strides=(1, 1, 1),
                                 padding='SAME',
                                 data_format='channels_last',
                                 activation=tf.nn.leaky_relu,
                                 dtype=tf.float16)
            self.convs.append(conv)
            
    def call(self, inputs):
        '''
            Input shape: (batch_size, frames, width, height, channels)
            
            Output shape: (batch_size, frames, width, height, filters)
        '''
        x = inputs
        for layer in self.convs:
            x = layer(x)
            
        return x

In [6]:
with tf.device('/GPU:0'):
    resblock = ResBLockConv3D(12, 5)
    resblock_result = resblock(tf.random.normal((32, 16, 512, 512, 3), dtype=tf.float16))
    resblock_result.shape

Executing op RandomStandardNormal in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/r

ResourceExhaustedError: OOM when allocating tensor with shape[32,12,16,512,512] and type half on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv3D]

In [6]:
class Encoder(keras.models.Model):
    def __init__(self, latent_dim):
        super(Encoder, self).__init__()
        self.conv1 = layers.Conv3D(filters=64,
                                   kernel_size=(4, 3, 3),
                                   strides=(2, 2, 2),
                                   padding='valid',
                                   data_format='channels_last',
                                   activation=tf.nn.leaky_relu,)
        self.conv2 = layers.Conv3D(filters=256,
                                   kernel_size=(3, 3, 3),
                                   strides=(2, 2, 2),
                                   padding='SAME',
                                   data_format='channels_last',
                                   activation=tf.nn.leaky_relu,)
        self.conv3 = layers.Conv3D(filters=512,
                                   kernel_size=(2, 3, 3),
                                   strides=(1, 2, 2),
                                   padding='SAME',
                                   data_format='channels_last',
                                   activation=tf.nn.leaky_relu,)
        self.conv4 = layers.Conv3D(filters=512,
                                   kernel_size=(2, 3, 3),
                                   strides=(1, 2, 2),
                                   padding='SAME',
                                   data_format='channels_last',
                                   activation=tf.nn.leaky_relu,)
        
    def call(self, inputs):
        ''' 
            Input shape: (batch, 16, 512, 512, 3)
            During process:
              conv1: (batch, 7, 255, 255, 32)
              conv2: (batch, 3, 128, 128, 64)
              conv3: (batch, 2, 128, 128, 64)
            Output shape: (batch, latent_dim)
        '''
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        return x